In [28]:
"""
fetch_prices_yahoo.py

Fetches full historical daily stock price data using yfinance and stores it in PostgreSQL.
"""

import os
import yfinance as yf
from datetime import datetime, UTC
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv
from database.price_data import Base, PriceData

# Load environment variables
load_dotenv()

# Configuration
DATABASE_URL = os.getenv("DATABASE_URL")

# SQLAlchemy setup
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

def fetch_yahoo_prices(ticker):
    """Fetch full historical data using yfinance and insert into PostgreSQL."""
    df = yf.download(ticker, start="1990-01-01", progress=False, auto_adjust=False)

    if df.empty:
        raise ValueError(f"No data returned for ticker {ticker}")

    records = []
    for date, row in df.iterrows():
        record = PriceData(
            ticker=ticker,
            date=date.date(),
            open=float(row['Open'].item()),
            high=float(row['High'].item()),
            low=float(row['Low'].item()),
            close=float(row['Close'].item()),
            adjusted_close=float(row['Adj Close'].item()),
            volume=int(row['Volume'].item()),
            source_api='Yahoo Finance',
            interval='1d',
            currency='USD',
            data_type='historical',
            created_at=datetime.now(UTC)
        )

        # Delete if duplicate exists
        session.query(PriceData).filter_by(
            ticker=record.ticker,
            date=record.date,
            interval=record.interval,
            source_api=record.source_api
        ).delete()

        records.append(record)

    session.bulk_save_objects(records)
    session.commit()
    print(f"Inserted {len(records)} price records for {ticker} from Yahoo Finance.")

if __name__ == '__main__':
    Base.metadata.create_all(engine)
    fetch_yahoo_prices('NVDA')  # Full history for NVIDIA


Inserted 6641 price records for NVDA from Yahoo Finance.
